In [3]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage

from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict, Annotated
import operator

# 1. LLM Setup
llm = ChatOllama(
    model="llama3.1:latest",
    temperature=0.0,
    base_url="http://localhost:11434",
    api_key="ollama"
)

# 2. State Schema
class AgentState(TypedDict):
    messages: Annotated[list, operator.add]
    should_exit: bool

# 3. User input node
def user_node(state: AgentState):
    user_input = input("You: ")
    if user_input.lower() in {"exit", "quit"}:
        print("Goodbye!")
        return {"should_exit": True}
    
    # Print the human input
    print(f"You: {user_input}")
    return {"messages": [HumanMessage(content=user_input)], "should_exit": False}

# 4. LLM response node
def llm_node(state: AgentState):
    last_user_msg = state["messages"][-1].content
    response = llm.invoke(state["messages"])
    print(f"Assistant: {response.content}")
    return {"messages": [response]}

# 5. Conditional routing function
def should_continue(state: AgentState):
    if state.get("should_exit", False):
        return END
    return "user"

# 6. Build LangGraph
graph = StateGraph(AgentState)
graph.add_node("user", user_node)
graph.add_node("agent", llm_node)

# Set entry point
graph.set_entry_point("user")

# Add conditional edges from user node
graph.add_conditional_edges(
    "user", 
    lambda state: END if state.get("should_exit", False) else "agent",
    {"agent": "agent", END: END}
)

# Add conditional edges from agent node  
graph.add_conditional_edges(
    "agent",
    should_continue,
    {"user": "user", END: END}
)

# 7. Compile and run
app = graph.compile()
final_state = app.invoke({"messages": [], "should_exit": False})

# Optional: Print a summary instead of the full state
# print(f"\nConversation ended. Total messages exchanged: {len(final_state['messages'])}")

You: hi
Assistant: How's it going? Is there something I can help you with or would you like to chat?
You: hey
Assistant: What's up? Want to talk about something in particular or just hang out for a bit?
Goodbye!


In [ ]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from typing_extensions import TypedDict
from typing import Annotated, Literal
import operator

In [5]:
# --- Define your tools ---

@tool
def llm_explain_like_poet(txt: str) -> str:
    """Explain a text or a concept in poetry."""
    prompt = f"Explain the following text as a poetry: {txt}"
    res = llm_main.invoke([HumanMessage(content=prompt)])
    return res.content

@tool
def llm_explain_like_pirate(txt: str) -> str:
    """Explain a text like a pirate."""
    prompt = f"Explain the following text like a pirate: {txt}"
    res = llm_main.invoke([HumanMessage(content=prompt)])
    return res.content

@tool
def llm_summarize_10words(txt: str) -> str:
    """Summarize a text down to 10 words."""
    # Create a proper prompt for summarization
    prompt = f"Summarize the following text in exactly 10 words: {txt}"
    res = llm_main.invoke([HumanMessage(content=prompt)])
    return res.content

@tool
def check_summary(text: str) -> bool:
    """Check if summary contains exactly 10 words."""
    return len(text.split()) == 10

In [6]:
explanation_tools = [llm_explain_like_poet, llm_explain_like_pirate]
explain_tool_node = ToolNode(explanation_tools)

summary_node = ToolNode([llm_summarize_10words])

check_node = ToolNode([check_summary])

In [7]:
tools = explanation_tools + [llm_summarize_10words, check_summary]

# Bind tools to the LLM
llm_with_tools = llm_main.bind_tools(tools)

# --- Define State Schema ---
class AgentState(TypedDict):
    """The state schema for our agent graph."""
    messages: Annotated[list, operator.add] # Messages will be appended to the list
    

# Define the agent node
def call_model(state: AgentState):
    messages = state['messages']
    response = llm_main.invoke(messages) #llm_with_tools.invoke(messages)
    return {"messages": [response]}

# Create the graph using the updated API
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("agent", call_model)

# Set entry point - updated syntax
workflow.add_edge(START, "agent")
workflow.add_edge("agent", "agent")
workflow.add_edge("agent", END)

In [8]:
app = workflow.compile()

In [10]:
app.invoke({
    "messages": [HumanMessage(content="Explain this text about LangGraph: LangGraph is a framework for building agentic applications with LLMs. LangGraph provides tools, chains, and agents to create complex workflows. It helps developers build context-aware, reasoning applications.")]
})

KeyboardInterrupt: 

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict, Annotated
import operator

# Define State
class AgentState(TypedDict):
    messages: Annotated[list, operator.add]

workflow = StateGraph(AgentState)

# User input node
def add_user_input(state: AgentState):
    user_input = input("You: ")
    print("*******")
    print("User input received:", user_input)
    if user_input.lower() in {"exit", "quit"}:
        print("Exiting conversation.")
        return END
    return {"messages": [{"type": "user", "content": user_input}]}

# Agent (LLM) node
def call_model(state: AgentState):
    messages = state['messages']
    response = llm_main.invoke(messages)
    print("*******")
    print("Assistant:", response.content)
    return {"messages": [response]}

# Add nodes
workflow.add_node("user", add_user_input)
workflow.add_node("agent", call_model)

# Routing
workflow.set_entry_point("user")
workflow.add_edge("user", "agent")
workflow.add_edge("agent", "user")
workflow.add_edge("user", END)  # loop

# Optional stop condition could go here

app = workflow.compile()
app.invoke({"messages": []})


*******
User input received: exit
Exiting conversation.


InvalidUpdateError: Expected dict, got __end__
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_GRAPH_NODE_RETURN_VALUE